In [3]:
# 스케쥴러에 필요한 모듈
import schedule 
import time 

# 크롤링에 필요한 모듈
import requests
import json
import urllib
import os 
import datetime as dt


# 수집된 뉴스 파일을 압축
import zipfile
import shutil

# 이메일 모듈

from sendmail import sendmail

In [4]:
def season_style():
    api_key = '384d0e4a284d03c77763c19a39f2e039'

    # 전달 요청 파라미터
    page = 1
    size =  80

    # 폴더 이름 생성
    datetime = dt.datetime.now().strftime('%y%m%d-%H%M%S')
    month = dt.datetime.now().month

    if month >= 3 and month <= 5:
        dirname = '%s_%s' % (datetime, '봄철코디')
        q = '봄철코디'
    elif month >= 6 and month <= 8:
        dirname = '%s_%s' % (datetime, '여름철코디')
        q = '여름철코디'
    elif month >= 9 and month <= 11:
        dirname = '%s_%s' % (datetime, '가을철코디')
        q = '가을철코디'
    else:
        dirname = '%s_%s' % (datetime, '겨울철코디')
        q = '겨울철코디'

    my_zip = zipfile.ZipFile("%s_%s.zip" % (datetime,q), 'w') 

    if not os.path.exists(dirname):
        os.mkdir(dirname)

    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36"
    header_info = {'User-agent': user_agent, 'referer': None, 'Authorization': 'KakaoAK ' + api_key}
    #header_info

    is_end = False

    image_list = []

    while not is_end:
        params = {'page':page, 'size':size, 'query':q}
        query = urllib.parse.urlencode(params)

        url_tpl = "https://dapi.kakao.com/v2/search/image"
        api_url = url_tpl + "?" + query
        api_url

        r = requests.get(api_url, headers=header_info)

        if r.status_code != 200:
            print("[%d Error] %s" % (r.status_code, r.reason))
            quit()

        r.encoding = "utf-8"
        image_dict = json.loads(r.text)
        image_dict

        is_end = image_dict['meta']['is_end']
        image_list += image_dict['documents']

        print("%dpage로부터 %d건 수집함 >>> 누적 데이터 수: %d" % (page, len(image_dict['documents']), len(image_list)))
        page += 1

        if page >= 2:
            is_end = True

    #image_list

    # 이미지 주소에 대해서만 반복
    for i, v in enumerate(image_list): 
        image_url = v['image_url']

        # 파일이 저장될 경로 생성
        path = "%s/%s_%s_%04d.jpg" % (dirname, datetime, q, i)
        print( "[%s] >> %s" % (path, image_url) )

        # 예외처리 구문 적용
        try:
            # 이미지 주소를 다운로드를 위해 stream 모드로 가져온다.
            r = requests.get(image_url, headers=header_info, stream=True)

            # 에러 발생시 저장이 불가능하므로 건너뛰고 반복의 조건식으로 이동
            if r.status_code != 200:
                print("##########> 저장실패 (%d)" % r.status_code)
                continue

            # 추출한 데이터를 저장
            # -> 'w': 텍스트 쓰기 모드, 'wb': 바이너리(이진값) 쓰기 모드
            with open(path, 'wb') as f:
                f.write(r.raw.read())
                print("----------> 저장성공")
                my_zip.write(path)
        except:
            print("~~~~~~~~~~~> 저장실패")
            continue
    my_zip.close()
    shutil.rmtree(dirname)
    
    # 메일 보내기
    today = dt.datetime.now()
    now_year = today.year
    now_month = today.month

    from_addr = 'cj562270@gmail.com'
    to_addr = 'leekh4232@kakao.com'

    subject_tpl = "{yy}년 {mm}월 %s 입니다." %q
   
    content_tpl = "{yy}년 {mm}월 %s." %q
   

    subject = subject_tpl.format(yy = now_year, mm = now_month)
    content = content_tpl.format(yy = now_year, mm = now_month)

    sendmail(from_addr,to_addr,subject,content,["%s_%s.zip" % (datetime,q)])

    print("메일 발송 완료 <%s>" %(to_addr))


In [5]:
# 1분에 한번씩 실행
schedule.every(1).minutes.do(season_style)

Every 1 minute do season_style() (last run: [never], next run: 2021-05-25 14:31:10)

In [ ]:
while True:    
    # 현재 시간에 동작해야 할 함수들을 찾아 구동함
    schedule.run_pending()
        
    # 1초 대기
    time.sleep(1)

1page로부터 80건 수집함 >>> 누적 데이터 수: 80
[210525-143110_봄철코디/210525-143110_봄철코디_0000.jpg] >> http://postfiles4.naver.net/20130322_83/xdadsof_13639480803635nVfy_JPEG/%BA%BD%BA%BD%BA%BD%C0%CC%BF%D4%BE%EE%BF%E4_%BA%BD%C3%B6%C4%DA%B5%F03.jpg?type=w2
----------> 저장성공
[210525-143110_봄철코디/210525-143110_봄철코디_0001.jpg] >> http://postfiles5.naver.net/20130322_164/xdadsof_1363948039944gqHHJ_JPEG/%BA%BD%BA%BD%BA%BD%C0%CC%BF%D4%BE%EE%BF%E4_%BA%BD%C3%B6%C4%DA%B5%F01.jpg?type=w2
----------> 저장성공
[210525-143110_봄철코디/210525-143110_봄철코디_0002.jpg] >> http://imgnews.naver.com/image/022/2008/03/19/20080319000885_0.jpg
----------> 저장성공
[210525-143110_봄철코디/210525-143110_봄철코디_0003.jpg] >> http://postfiles13.naver.net/MjAxOTAyMjBfMTE0/MDAxNTUwNjQyMTk3ODA3.Ks2OOIyy8abxQYKoAxXzkdmHGfT2qMbnrYZQnvil_iYg.ITbiblgtK7TADdz_chrnyk0fjrBaMl5jdkSrvuIEgKEg.JPEG.ytrim01245/pc002000488.jpg?type=w966
----------> 저장성공
[210525-143110_봄철코디/210525-143110_봄철코디_0004.jpg] >> http://img.ezday.co.kr/cache/board/2008/03/27/c6b1b5a92e868f3cfe1